<a href="https://colab.research.google.com/github/labadilloj/TLG-DS-2025/blob/main/DS_C2_SC2_LESLIE_BADILLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Análisis del progreso mundial de vacunación##


In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()  # seleccionar archivo .csv


a. Extraer la información del archivo.

In [ ]:
df = pd.read_csv("country_vaccinations.csv")
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'country_vaccinations.csv'

b. Mostrar la estructura y tipos de datos de cada columna para identificar qué operaciones puedes realizar con cada una de ellas, asegurándote que las columnas con fechas sean del tipo datetime64.

In [ ]:
print(df.dtypes) #tipo de datp
df["date"] = pd.to_datetime(df["date"], errors="coerce") #cambbio de data
print(df["date"].dtype)   # debería salir datetime64[ns]
print(df["date"])

c. Determinar la cantidad de vacunas aplicadas de cada compañía (con base en cómo lo reporta cada país en la columna vaccines, en otras palabras, agrupe por vaccines y realice la sumatoria).

In [ ]:
resumen_vaccines = (
    df.groupby('vaccines', as_index=False)['daily_vaccinations']
      .sum()
      .sort_values('daily_vaccinations', ascending=False)
)

resumen_vaccines['daily_vaccinations'] = (
    resumen_vaccines['daily_vaccinations'].round(0).astype('int64')
)

print("\nCantidad de vacunas por compañía:")
print(resumen_vaccines.head(25))  # muestra las primeras 20 filas

d. Obtener la cantidad de vacunas aplicadas en todo el mundo.

In [ ]:
total_global_daily = df['daily_vaccinations'].sum()
print(f"Total de vacunas: {int(total_global_daily):,}")

e. Calcular el promedio de vacunas aplicadas por país.

In [ ]:
promedio_por_pais = (
    df.groupby('country', as_index=False)['daily_vaccinations']
      .mean()                                  # promedio diario
      .rename(columns={'daily_vaccinations':'promedio_diario'})
      .sort_values('promedio_diario', ascending=False)
)

promedio_por_pais['promedio_diario'] = (
    promedio_por_pais['promedio_diario'].round(0).astype('int64')
)

print("Promedio diario de vacunas aplicadas por país:")
print(promedio_por_pais.head(20))  # muestra los 20 mayores


f. Determinar la cantidad de vacunas aplicadas el día 29/01/21 en todo el mundo.

In [ ]:
#print(df['date'])
f = pd.Timestamp("2021-01-29")
filtro_fecha = df.loc[df['date'] == f].copy()

total_dia = pd.to_numeric(filtro_fecha['daily_vaccinations']).sum()
print(f"Total de vacunas aplicadas el {f.date()}: {int(total_dia):,}")

g. Crear un dataframe nuevo denominado conDiferencias que contenga los datos originales y una columna derivada (diferencias) con las diferencias de aplicación entre las columnas daily_vaccionations y daily_vaccionations_raw.  

In [ ]:
conDiferencias = df.copy()          # copia del original
conDiferencias['diferencias'] = conDiferencias['daily_vaccinations'] - conDiferencias['daily_vaccinations_raw']
print(conDiferencias[['daily_vaccinations', 'daily_vaccinations_raw', 'diferencias']])
#print(conDiferencias.head())  #Revisión de data completa

h. Obtener el periodo de tiempo entre el registro con fecha más reciente y el registro con fecha más antigua.

In [ ]:
primera_fecha = df['date'].min()
ultima_fecha   = df['date'].max()

print("Más antigua:", primera_fecha)
print("Más reciente:", ultima_fecha)

rango = ultima_fecha - primera_fecha
print("Días:", rango.days)

i. Crear un dataframe nuevo denominado conCantidad que contenga los datos originales y una columna derivada (canVac) con la cantidad de vacunas utilizadas cada día (usar la columna vaccines y separar por el carácter , ).

In [ ]:
conCantidad = df.copy()          # copia del original
conCantidad['canVac'] = conCantidad['vaccines'].str.split(', ').str.len()
print(conCantidad[['vaccines', 'canVac']])
#print(conCantidad.head())  #Revisión de data completa

j. Generar un dataframe denominado antes20 con todos los registros que se hayan realizado antes del 20 de diciembre de 2020.

In [ ]:
antes20 = df.copy()          # copia del original
antes20 = antes20[antes20['date'] < '2020-12-20']
print(antes20)


k. Obtener un dataframe denominado pfizer con todos los registros donde se haya utilizado la vacuna Pfizer.

In [ ]:
pfizer = df.copy()        # copia del original
col = 'tipo_vacuna' if 'tipo_vacuna' in df.columns else 'vaccines'

pfizer = df.loc[
    df[col].str.contains('Pfizer/BioNTech', case=False, na=False, regex=False)
].copy()

print(pfizer.head())
total_filas = len(pfizer)
print("Total de filas:", total_filas)


l. Almacenar los dataframes generados (conDiferencias, conCantidad, antes20 y pfizer) en un archivo de Excel denominado resultadosReto.xlsx, donde cada dataframe ocupe una hoja diferente. Se recomienda ver la documentación de pd.ExcelWriter.

In [ ]:
# Guarda cada DataFrame en una hoja distinta del mismo archivo Excel
with pd.ExcelWriter("resultadosReto.xlsx", engine="openpyxl") as writer:
    conDiferencias.to_excel(writer, sheet_name="conDiferencias", index=False)
    conCantidad.to_excel(writer,    sheet_name="conCantidad",   index=False)
    antes20.to_excel(writer,        sheet_name="antes20",       index=False)
    pfizer.to_excel(writer,         sheet_name="pfizer",        index=False)

print("Archivo 'resultadosReto.xlsx' creado con 4 hojas")